Import Libraries 

In [13]:
import torch
import torch.nn as nn
import math

Positional Encoding 

In [14]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        self.max_len = max_len
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)  # even indices
        pe[:, 1::2] = torch.cos(position * div_term)  # odd indices
        pe = pe.unsqueeze(0)  # shape: (1, max_len, d_model)
        self.register_buffer('pe', pe)

    def forward(self, x):
        # x shape: (batch_size, seq_len, d_model)
        return x + self.pe[:, :x.size(1)]


Masked Multi-Head self-attention 

In [15]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        assert d_model % num_heads == 0
        self.d_k = d_model // num_heads
        self.num_heads = num_heads

        self.qkv_proj = nn.Linear(d_model, 3 * d_model)
        self.o_proj = nn.Linear(d_model, d_model)

    def forward(self, x, mask=None):
        B, T, C = x.shape
        qkv = self.qkv_proj(x)  # shape: (B, T, 3C)
        qkv = qkv.reshape(B, T, self.num_heads, 3 * self.d_k)
        qkv = qkv.permute(2, 0, 1, 3).chunk(3, dim=-1)
        q, k, v = qkv  # each has shape: (num_heads, B, T, d_k)

        attn_scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, float('-inf'))

        attn_weights = torch.softmax(attn_scores, dim=-1)
        out = torch.matmul(attn_weights, v)  # shape: (num_heads, B, T, d_k)
        out = out.transpose(0, 1).contiguous().view(B, T, C)  # merge heads
        return self.o_proj(out)


Feed-Forward 

In [16]:
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.ReLU(),
            nn.Linear(d_ff, d_model)
        )

    def forward(self, x):
        return self.net(x)


Decoder 

In [17]:
class DecoderBlock(nn.Module):
    def __init__(self, d_model, num_heads, d_ff):
        super().__init__()
        self.attn = MultiHeadAttention(d_model, num_heads)
        self.ff = FeedForward(d_model, d_ff)
        self.ln1 = nn.LayerNorm(d_model)
        self.ln2 = nn.LayerNorm(d_model)

    def forward(self, x, mask):
        x = self.ln1(x + self.attn(x, mask))
        x = self.ln2(x + self.ff(x))
        return x


Casual Mask 

In [ ]:
def generate_causal_mask(seq_len):
    mask = torch.tril(torch.ones(seq_len, seq_len)).unsqueeze(0).unsqueeze(1)
    return mask  # shape: (1, 1, seq_len, seq_len)


Transformer Decoder 

In [ ]:
class TransformerDecoder(nn.Module):
    def __init__(self, vocab_size, d_model=512, num_heads=8, d_ff=2048, num_layers=6, max_len=512):
        super().__init__()
        self.token_embed = nn.Embedding(vocab_size, d_model)
        self.pos_embed = PositionalEncoding(d_model, max_len)
        self.layers = nn.ModuleList([DecoderBlock(d_model, num_heads, d_ff) for _ in range(num_layers)])
        self.norm = nn.LayerNorm(d_model)
        self.lm_head = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        B, T = x.shape
        mask = generate_causal_mask(T).to(x.device)
        x = self.token_embed(x)
        x = self.pos_embed(x)

        for layer in self.layers:
            x = layer(x, mask)

        x = self.norm(x)
        return self.lm_head(x)  # output logits


Actual Inference

In [30]:
@torch.no_grad()
def generate_text(model, start_tokens, max_new_tokens=20, eos_token_id=None):
    model.eval()
    generated = start_tokens.clone()

    for _ in range(max_new_tokens):
        if generated.size(1) > model.pos_embed.pe.size(1):
            generated = generated[:, -model.pos_embed.pe.size(1):]
        logits = model(generated)  # shape: (B, T, vocab_size)
        next_logits = logits[:, -1, :]  
        next_token = torch.argmax(next_logits, dim=-1, keepdim=True)  
        generated = torch.cat([generated, next_token], dim=1)
        if eos_token_id is not None:
            if torch.all(next_token == eos_token_id):
                break

    return generated

In [39]:
# Example input (batch size 1)
start = torch.tensor([[1, 5, 7]], dtype=torch.long)
output = generate_text(model, start, max_new_tokens=10)
print("Generated token IDs:", output.tolist())

Generated token IDs: [[1, 5, 7, 51, 29, 61, 32, 55, 41, 36, 14, 88, 88]]
